<a href="https://colab.research.google.com/github/RAGAVARSHINI03/Recurrent-Neural-Network/blob/main/Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem Statement1: Build a text classification RNN ( Recurrent Neural Network) model using IMDB dataset.

Import the IMDB dataset using tensorflow_datasets and perform the following tasks.

Tasks to be performed:

•Import the required libraries

•Shuffle the data for training and create batches of text and label pairs

•Encode the text data Note: The simplest way to process text for training is using the Text Vectorization layer.

•Build a sequential model using tf.keras.Sequential function

•Compile the model

•Train the model using train dataset

•Test the data using tset dataset and evaluate the model by passing a sentence

Note:If the prediction is >= 0.0, it is positive else it is negative

In [2]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, SimpleRNN, Dense
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds

In [3]:
# Load the IMDB dataset
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.0BZK3W_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.0BZK3W_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.0BZK3W_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
# Shuffle and batch the data
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [5]:
# Encode the text data using TextVectorization
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

vectorizer = TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=MAX_SEQUENCE_LENGTH)

In [6]:
# Adapt the vectorizer to the training dataset
train_text = train_data.map(lambda x, y: x)
vectorizer.adapt(train_text)

In [7]:
# Build the Sequential RNN model
model = Sequential([
    vectorizer,  # Text Vectorization layer
    Embedding(VOCAB_SIZE, 64),  # Embedding layer
    SimpleRNN(64, return_sequences=False),  # Simple RNN layer
    Dense(64, activation='relu'),  # Dense layer with ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [8]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [9]:
# Train the model using the train dataset
EPOCHS = 5
history = model.fit(train_data, epochs=EPOCHS, validation_data=test_data)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 29s 64ms/step - accuracy: 0.5004 - loss: 0.6959 - val_accuracy: 0.5033 - val_loss: 0.6947
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 39s 60ms/step - accuracy: 0.5182 - loss: 0.6942 - val_accuracy: 0.5118 - val_loss: 0.6961
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 45s 70ms/step - accuracy: 0.5307 - loss: 0.6923 - val_accuracy: 0.5259 - val_loss: 0.6906
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 37s 61ms/step - accuracy: 0.6367 - loss: 0.6445 - val_accuracy: 0.6671 - val_loss: 0.6212
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 27s 70ms/step - accuracy: 0.7046 - loss: 0.5694 - val_accuracy: 0.5964 - val_loss: 0.6868


In [10]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.5920 - loss: 0.6893
Test Loss: 0.68675696849823
Test Accuracy: 0.5964000225067139


In [12]:
# Test the model with a sample sentence
def predict_sentiment(model, sentence):
    sentence_tensor= tf.constant([sentence])
    prediction = model.predict([sentence_tensor])[0][0]
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    print(f"Sentence: {sentence}")
    print(f"Predicted Sentiment: {sentiment} (Score: {prediction:.4f})")

# Example test sentence
test_sentence = "The movie was absolutely fantastic, I loved it!"
predict_sentiment(model, test_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Sentence: The movie was absolutely fantastic, I loved it!
Predicted Sentiment: Negative (Score: 0.4714)
